# 如何从工具访问 RunnableConfig

:::info 先决条件

本指南假设您熟悉以下概念：

- [LangChain 工具](/docs/concepts/tools)
- [自定义工具](/docs/how_to/custom_tools)
- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)
- [配置可运行对象的行为](/docs/how_to/configure/)

:::

如果您有一个调用 [聊天模型](/docs/concepts/chat_models/)、[检索器](/docs/concepts/retrievers/) 或其他 [可运行对象](/docs/concepts/runnables/) 的[工具](/docs/concepts/tools/)，您可能希望访问这些可运行对象的内部事件或用其他属性配置它们。本指南将向您展示如何正确手动传递参数，以便您可以使用 `astream_events()` 方法执行此操作。

工具是[可运行对象](/docs/concepts/runnables/)，您可以在接口层面将它们与其他可运行对象一样对待——您可以像平常一样调用 `invoke()`、`batch()` 和 `stream()`。但是，在编写自定义工具时，您可能希望调用聊天模型或检索器等其他可运行对象。为了正确跟踪和配置这些子调用，您需要手动访问并传入工具当前的 [`RunnableConfig`](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.config.RunnableConfig.html) 对象。本指南将展示一些如何执行此操作的示例。

:::caution 兼容性

本指南需要 `langchain-core>=0.2.16`。

:::

## 通过参数类型推断

要从自定义工具中访问引用活动的配置对象，您需要向工具的签名添加一个类型为 `RunnableConfig` 的参数。当您调用工具时，LangChain 将检查您的工具签名，查找类型为 `RunnableConfig` 的参数，如果存在，则会用正确的值填充该参数。

**注意：** 参数的实际名称并不重要，只有类型才重要。

为了说明这一点，请定义一个接受两个参数的自定义工具——一个类型为字符串，另一个类型为 `RunnableConfig`：

In [ ]:
%pip install -qU langchain_core

In [2]:
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool


@tool
async def reverse_tool(text: str, special_config_param: RunnableConfig) -> str:
    """A test tool that combines input text with a configurable parameter."""
    return (text + special_config_param["configurable"]["additional_field"])[::-1]

接着，如果我们使用包含 `configurable` 字段的 `config` 来调用该工具，我们就能看到 `additional_field` 已被正确传递：

In [3]:
await reverse_tool.ainvoke(
    {"text": "abc"}, config={"configurable": {"additional_field": "123"}}
)

'321cba'

## 下一步

您现在已经了解了如何在工具中配置和流式传输事件。接下来，请参阅以下指南，了解更多关于使用工具的信息：

- [从自定义工具中的子运行流式传输事件](/docs/how_to/tool_stream_events/)
- 将[工具结果传回模型](/docs/how_to/tool_results_pass_to_model)

您还可以查看一些更具体的工具调用用法：

- 构建[使用工具的链和代理](/docs/how_to#tools)
- 从模型获取[结构化输出](/docs/how_to/structured_output/)